In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation
from tensorflow.keras import backend as K
import numpy as np
import os

num_classes = 10
img_rows, img_cols = 28, 28
 
# 通过Keras封装好的API加载MNIST数据。其中trainX就是一个60000 * 28 * 28的数组，
# trainY是每一张图片对应的数字。
def load_data(path='/Users/zhangjrjlu/datasets/mnist.npz'):       #####该部分是对于.npz(numpy格式)文件的读取方法
    f = np.load(path)
    x_train, y_train = f['x_train'], f['y_train']
    x_test, y_test = f['x_test'], f['y_test']
    f.close()
    return (x_train, y_train), (x_test, y_test)
 
# the data, split between train and test sets
(trainX, trainY), (testX, testY) = load_data()

# 根据对图像编码的格式要求来设置输入层的格式。需要根据对于图像格式的要求来判断
if K.image_data_format() == 'channels_first':     
    trainX = trainX.reshape(trainX.shape[0], 1, img_rows, img_cols)   #四维数组，第一项是图片的数量，表示第几张图片
    testX = testX.reshape(testX.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    trainX = trainX.reshape(trainX.shape[0], img_rows, img_cols, 1)
    testX = testX.reshape(testX.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
trainX = trainX.astype('float32')
testX = testX.astype('float32')
trainX /= 255.0
testX /= 255.0
 
# 将标准答案转化为需要的格式（one-hot编码）。
trainY = keras.utils.to_categorical(trainY, num_classes)
testY = keras.utils.to_categorical(testY, num_classes)
print(trainY.shape)
"""
我们知道彩色图像一般会有Width, Height, Channels，而“channels_first”或“channels_last”，则代表数据的通道维的位置。
该参数是Keras 1.x中的image_dim_ordering，“channels_last”对应原本的“tf”，“channels_first”对应原本的“th”。
以128x128的数据为例，“channels_first”应将数据组织为（3,128,128），而“channels_last”应将数据组织为（128,128,3）。
该参数的默认值是~/.keras/keras.json中设置的值，若从未设置过，则为“channels_last”。


keras.utils.to_categorical(y, num_classes=None, dtype='float32')
将整型的类别标签转为onehot编码。y为int数组，num_classes为标签类别总数
"""
"""
灰度数据表示有两种方法：
1、 uint8类型
2、double类型
其中uint8类型数据的取值范围为 [0,255]，而double类型数据的取值范围为[0,1]，两者正好相差255倍
对于double类型数据，其取值大于1时，就会表示为白色，不能显示图像的信息，故当运算数据类型为double时，为了显示图像要除255。
"""

(60000, 10)


'\n灰度数据表示有两种方法：\n1、 uint8类型\n2、double类型\n其中uint8类型数据的取值范围为 [0,255]，而double类型数据的取值范围为[0,1]，两者正好相差255倍\n对于double类型数据，其取值大于1时，就会表示为白色，不能显示图像的信息，故当运算数据类型为double时，为了显示图像要除255。\n'

In [4]:
model = Sequential()
##需要先定义一个Sequential类，然后在Sequential的实例中定义网络结构
model.add(Conv2D(32, kernel_size=[5, 5], input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), activation=tf.nn.relu))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())###把上层的输出拉平
model.add(Dense(500, activation=tf.nn.relu))
model.add(Dense(num_classes, activation='softmax'))

#model.build(input_shape=(60000, 1,28,28))
model.summary()
 
# 定义损失函数、优化函数和评测方法。
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(),
              metrics=['accuracy'])
"""
keras.layers.Conv2D(filters, kernel_size,strides=(1, 1), padding='valid', data_format=None, dilation_rate=(1, 1), activation=None, 
                     use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, 
                     bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)
filters: 整数，输出空间的维度 （即卷积中滤波器的数量）。
kernel_size: 一个整数，或者 2 个整数表示的元组或列表， 指明 2D 卷积窗口的宽度和高度。 可以是一个整数，为所有空间维度指定相同的值。
strides: 一个整数，或者 2 个整数表示的元组或列表， 指明卷积沿宽度和高度方向的步长。 可以是一个整数，为所有空间维度指定相同的值。
padding: "valid" 或 "same" (大小写敏感)。   valid padding就是不padding，而same padding就是指padding完尺寸与原来相同
图像识别一般来说都要padding，尤其是在图片边缘的特征重要的情况下。padding多少取决于我们需要的输出是多少
data_format: 字符串， channels_last (默认) 或 channels_first 之一.
activation: 要使用的激活函数 (详见 activations)。 如果你不指定，则不使用激活函数 (即线性激活： a(x) = x)。
use_bias: 布尔值，该层是否使用偏置向量。
kernel_initializer: kernel 权值矩阵的初始化器 (详见 initializers)。
bias_initializer: 偏置向量的初始化器 (详见 initializers)。
kernel_regularizer: 运用到 kernel 权值矩阵的正则化函数 (详见 regularizer)。
bias_regularizer: 运用到偏置向量的正则化函数 (详见 regularizer)。
activity_regularizer: 运用到层输出（它的激活值）的正则化函数 (详见 regularizer)。
kernel_constraint: 运用到 kernel 权值矩阵的约束函数 (详见 constraints)。
bias_constraint: 运用到偏置向量的约束函数 (详见 constraints)。
"""

"""
keras.layers.pooling.MaxPooling1D(pool_size=2, strides=None, padding='valid）参数同Conv2D
keras.layers.Dense(units, 
				  activation=None, 
				  use_bias=True, 
				  kernel_initializer='glorot_uniform', 
				  bias_initializer='zeros', 
				  kernel_regularizer=None, 
				  bias_regularizer=None, 
			      activity_regularizer=None, 
				  kernel_constraint=None, 
				  bias_constraint=None)
参数同Conv2D，其中units表示该全连接层中有几个节点

keras.model.compile(loss='目标函数', optimizer=optimizer, metrics=['accuracy'])
loss为想要使用的损失函数
optimizer为想要进行优化的方法
metrics为评估方法
"""

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
activation (Activation)      (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 500)              

"\nkeras.layers.pooling.MaxPooling1D(pool_size=2, strides=None, padding='valid）参数同Conv2D\nkeras.layers.Dense(units, \n\t\t\t\t  activation=None, \n\t\t\t\t  use_bias=True, \n\t\t\t\t  kernel_initializer='glorot_uniform', \n\t\t\t\t  bias_initializer='zeros', \n\t\t\t\t  kernel_regularizer=None, \n\t\t\t\t  bias_regularizer=None, \n\t\t\t      activity_regularizer=None, \n\t\t\t\t  kernel_constraint=None, \n\t\t\t\t  bias_constraint=None)\n参数同Conv2D，其中units表示该全连接层中有几个节点\n\nkeras.model.compile(loss='目标函数', optimizer=optimizer, metrics=['accuracy'])\nloss为想要使用的损失函数\noptimizer为想要进行优化的方法\nmetrics为评估方法\n"

In [5]:
model.fit(trainX, trainY,
          batch_size=128,
          epochs=10,
          validation_data=(testX, testY))  
#fit( x, y, batch_size=32, epochs=10, verbose=1, callbacks=None, validation_split，validation_data=None）
#validation_split：0~1之间的浮点数，用来指定训练集的一定比例数据作为验证集。验证集将不参与训练，并在每个epoch结束后测试的模型的指标，
#如损失函数、精确度等。注意，validation_split的划分在shuffle之前，因此如果你的数据本身是有序的，需要先手工打乱再指定validation_split，
#否则可能会出现验证集样本不均匀。

# 在测试数据上计算准确率。
score = model.evaluate(testX, testY)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
"""

model.evaluate
输入数据和标签,输出损失和精确度.
	# 评估模型,不输出预测结果
	loss,accuracy = model.evaluate(X_test,Y_test)
	print('\ntest loss',loss)
	print('accuracy',accuracy)
    
model.predict
输入测试数据,输出预测结果
(通常用在需要得到预测结果的时候)
    #模型预测,输入测试集,输出预测结果
    y_pred = model.predict(X_test,batch_size = 1)
"""

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
43648/60000 [====================>.........] - ETA: 3s - loss: 1.3684 - accuracy: 0.6492

KeyboardInterrupt: 